In [1]:
import pandas as pd
import geopandas as gpd
import folium
from json import loads, dumps
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from shapely.geometry import Point
import time
import googlemaps
import osmnx as ox
from sklearn.cluster import KMeans
import numpy as np
import constants

In [3]:
df = pd.read_csv('ibsj_sector.csv')
df = df.drop(columns=['prefix', 'name', 'city', 'state'])

null_address = len(df.loc[df['address_1'].isnull()])
print("El archivo tiene {} registros".format(len(df)))
#print("Hay {} direcciones vacias en el archivo".format(null_address))

df = df[df['address_1'].notna()]
df = df[df['address_1'] != 'Actualizar']
df['address_1'] = df['address_1'].replace({'#':'', 'C/':'Calle '}, regex=True)
df = df.reset_index()
print("Luego de eliminar los registros sin direccion, quedan {} registros".format(len(df)))

El archivo tiene 771 registros
Luego de eliminar los registros sin direccion, quedan 656 registros


In [ ]:
# Configurar la API de Google Maps
API_KEY = constants.API_KEY  
gmaps = googlemaps.Client(key=API_KEY)

# Función para obtener coordenadas y datos extra
def obtener_datos_direccion(direccion):
    try:
        geocode_result = gmaps.geocode(direccion, components={"country": "DO"})  # Restringido a República Dominicana
        if geocode_result:
            location = geocode_result[0]
            lat = location["geometry"]["location"]["lat"]
            lng = location["geometry"]["location"]["lng"]
            address_components = {comp["types"][0]: comp["long_name"] for comp in location["address_components"]}

            return pd.Series({
                "geometry": Point(lng, lat),
                "lng":lng,
                "lat":lat,
                "calle": address_components.get("route", ""),
                "numero": address_components.get("street_number", ""),
                "barrio": address_components.get("sublocality", address_components.get("political", address_components.get("sublocality_level_1", ""))),
                "ciudad": address_components.get("locality", address_components.get("administrative_area_level_2", "")),
                "estado": address_components.get("administrative_area_level_1", ""),
                "pais": address_components.get("country", ""),
                "codigo_postal": address_components.get("postal_code", "")
            })
    except Exception as e:
        print(f"Error con {direccion}: {e}")

    # Si falla, devolver valores vacíos
    return pd.Series({"geometry": None, "lng":"", "lat":"", "calle": "", "numero": "", "barrio": "", "ciudad": "", "estado": "", "pais": "", "codigo_postal": ""})


# Aplicar la función a cada dirección
df_info = df["address_1"].apply(obtener_datos_direccion)

# Unir los datos con el dataframe original
df = pd.concat([df, df_info], axis=1)

# Convertir a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Extraer las coordenadas (latitud y longitud)
coordinates = np.array([(point.y, point.x) for point in gdf.geometry])

# Aplicar KMeans para clasificar los puntos en clusters
kmeans = KMeans(n_clusters=8)  # Cambia el número de clusters (5 en este caso)
gdf['cluster'] = kmeans.fit_predict(coordinates)

# Busca y asigna las provincias a los puntos
provincias = ox.features_from_place(
    "Ozama,  Dominican Republic",
    {"boundary": "administrative","admin_level": "6"}
)

provincias = provincias[provincias['name'].isin(
            ['Pedro Brand','Santo Domingo Norte', 
             'Distrito Nacional', 'Boca Chica',           
             'Los Alcarrizos','San Antonio de Guerra',
             'San Luis','Santo Domingo Este',
             'Santo Domingo Norte','Santo Domingo Oeste']
)]

# Agregar cada polígono de provincia al mapa
# for _, row in provincias.iterrows():
#     if row.geometry is not None:
#         folium.GeoJson(row.geometry, tooltip=row.get("name")).add_to(mapa)

gdf["poligono"] = gdf.geometry.apply(
    lambda punto: provincias[provincias.contains(punto)]["name"].values[0] 
    if not provincias[provincias.contains(punto)].empty else "Fuera de cualquier polígono"
)


In [5]:
import sqlite3

con = sqlite3.connect("geolocation.sqlite")

gdf.to_sql("data", con)

ProgrammingError: Error binding parameter 8: type 'Point' is not supported

In [ ]:

def get_cluster_color(cluster_label):
    colors = [
        'purple', 'black', 'cadetblue', 'pink', 'red', 'blue', 'darkgreen', 
        'darkred', 'lightgreen', 'orange', 'beige', 'darkpurple', 'darkblue', 'green', 'gray', 'lightgray', 'lightred', 'lightblue'
    ]
    return colors[cluster_label]

# Asegurar que las coordenadas estén en formato correcto
# df["geometry"] = gpd.points_from_xy(df["geometry"].apply(lambda x: x.x), df["geometry"].apply(lambda x: x.y))
# gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Crear un mapa centrado en República Dominicana
mapa = folium.Map(location=[18.7357, -70.1627], zoom_start=9)

# Agregar puntos al mapa con información detallada
for idx, row in gdf.iterrows():
    popup_text = f"""
    <b>Dirección Original:</b> {row["address_1"]}<br>
    <b>Nombre:</b> {row["first_name"]}<br>
    <b>Apellido:</b> {row["last_name"]}<br>
    <b>Calle:</b> {row["calle"]}<br>
    <b>Número:</b> {row["numero"]}<br>
    <b>Barrio:</b> {row["barrio"]}<br>
    <b>Ciudad:</b> {row["poligono"]}<br>
    <b>País:</b> {row["pais"]}<br>
    <b>Código Postal:</b> {row["codigo_postal"]}<br>
    <b>Cluster:</b> {row["cluster"]}
    """

    marker_color = get_cluster_color(row['cluster'])
    
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],  # Latitud, Longitud
        popup=folium.Popup(popup_text, max_width=300),  # Información al hacer clic
        tooltip=row["address_1"],  # Muestra la dirección al pasar el mouse
        icon=folium.Icon(color=marker_color, icon="info-sign")  # Color y estilo del marcador
    ).add_to(mapa)

# Guardar el mapa como HTML y mostrarlo
# mapa.save("mapa_interactivo.html")

# mapa


In [ ]:
gdf.to_csv('data.csv', index=False)

In [34]:
gdf_load = pd.read_csv('data.csv')
gdf_load



,Unnamed: 0,index,id,first_name,last_name,address_1,address_2,geometry,lng,lat,calle,numero,barrio,ciudad,estado,pais,codigo_postal,cluster,poligono
0,0,0,106123565,José Manuel,Pablo Michelen,Calle Freddy Gatón Arce 9,Viejo Arroyo Hondo,POINT (-69.9422292 18.4890069),-69.942229,18.489007,Calle Freddy Gatón Arce,9,Viejo Arroyo Hondo,Santo Domingo,Distrito Nacional,Dominican Republic,10510.0,5,Distrito Nacional
1,1,1,107350252,Cirilo,Camacho,Ave. Puerta de Hierro No. 42,Puerta de Hierro,POINT (-69.96908640000001 18.5304113),-69.969086,18.530411,Avenida Puerta de Hierro,42,Arroyo Manzano,Santo Domingo,Distrito Nacional,Dominican Republic,NaN,1,Distrito Nacional
2,2,2,107350269,Alfredo,Camacho,Hector Garcia Godoy No 24,NaN,POINT (-69.9384001 18.4990717),-69.938400,18.499072,Hector Garcia Godoy,NaN,Viejo Arroyo Hondo,Santo Domingo,Distrito Nacional,Dominican Republic,NaN,5,Distrito Nacional
3,3,3,107350280,Ángel Martín,Castillo Franco,"Calle D, 8, Viejo Arroyo Hondo",NaN,POINT (-69.9389753 18.4876265),-69.938975,18.487627,Calle D,8,Viejo Arroyo Hondo,Santo Domingo,Distrito Nacional,Dominican Republic,NaN,5,Distrito Nacional
4,4,4,107350285,Celso,Cepeda,Residencial Palmas de las Praderas,Palma de las Praderas,POINT (-70.0159017 18.5075192),-70.015902,18.507519,NaN,NaN,NaN,Santo Domingo,Santo Domingo Province,Dominican Republic,NaN,4,Los Alcarrizos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,651,763,154666924,Mario,Figoli,"Res. Poniente, Av. Pdte. Jacobo Majluta",E-102,POINT (-69.9182304 18.5466242),-69.918230,18.546624,Avenida Presidente Jacobo Majluta Azar,NaN,NaN,NaN,NaN,Dominican Republic,NaN,3,Santo Domingo Norte
652,652,764,155872196,Susana,Perez,"Manzana 4687, edif.13 ap 1-b, Invivienda",NaN,POINT (-69.8215129 18.5072147),-69.821513,18.507215,NaN,NaN,Invivienda,Santo Domingo Este,Santo Domingo,Dominican Republic,NaN,7,Santo Domingo Este
653,653,766,156660627,Joel,Jean-Louis,Calle Juan A. Minaya No. 16,Ensanche Miraflores,POINT (-69.9095562 18.4768533),-69.909556,18.476853,Calle Juan A. Minaya,16,Miraflores,Santo Domingo,Distrito Nacional,Dominican Republic,10203.0,2,Distrito Nacional
654,654,769,161772182,Bacilia,Montaño de Sánchez,"Av. Monumental No. 6, Los Girasoles","Residencial Parque Verde, Bloque C, Apto 203",POINT (-69.989584 18.5155984),-69.989584,18.515598,Avenida Monumental,6,Altos De Arroyo Hondo,Santo Domingo,Distrito Nacional,Dominican Republic,NaN,4,Distrito Nacional


In [43]:
gdf_load.loc[gdf_load['lng'] == '0']

,Unnamed: 0,index,id,first_name,last_name,address_1,address_2,geometry,lng,lat,calle,numero,barrio,ciudad,estado,pais,codigo_postal,cluster,poligono


In [49]:
geometry = [Point(xy) for xy in zip(gdf_load["lng"], gdf_load["lat"])]  # Crea los puntos
gdf_load = gpd.GeoDataFrame(gdf_load, geometry=geometry, crs="EPSG:4326")  # Asigna CRS WGS84

In [6]:
gdf.loc[gdf['poligono'] == 'Fuera de cualquier polígono'].to_csv('arreglar.csv',index=False)

In [7]:
gmaps = googlemaps.Client(key=API_KEY)
direccion = "Calle López Contreras 6, Herrera"  # Reemplázala con una dirección específica
geocode_result = gmaps.geocode(direccion, components={"country": "DO"})

location = geocode_result[0]
lat = location["geometry"]["location"]["lat"]
lng = location["geometry"]["location"]["lng"]
address_components = {comp["types"][0]: comp["long_name"] for comp in location["address_components"]}


df_test = pd.DataFrame(data={'lat': [lat], 'lng': [lng]})
df_test['geometry'] = Point(lng, lat)

mapa = folium.Map(location=[18.7357, -70.1627], zoom_start=9)
folium.Marker(
           location=[df_test.loc[0,'geometry'].y, df_test.loc[0,'geometry'].x]
        ).add_to(mapa)

mapa

In [ ]:

import geopandas as gpd
import folium

# Descargar los polígonos de los barrios de Santo Domingo
gdf_barrios = ox.features_from_place(
    "Ozama,  Dominican Republic",
    {"boundary": "administrative","admin_level": "6"}
)

gdf_barrios = gdf_barrios[gdf_barrios['name'].isin(
            ['Pedro Brand','Santo Domingo Norte', 
             'Distrito Nacional', 'Boca Chica',           
             'Los Alcarrizos','San Antonio de Guerra',
             'San Luis','Santo Domingo Este',
             'Santo Domingo Norte','Santo Domingo Oeste']
)]

# Crear un mapa centrado en Santo Domingo
mapa = folium.Map(location=[18.5, -69.9], zoom_start=9)

# Agregar cada polígono al mapa
for _, row in gdf_barrios.iterrows():
    if row.geometry is not None:
        folium.GeoJson(row.geometry, tooltip=row.get("name")).add_to(mapa)

# Mostrar el mapa
mapa.save("mapa_barrios_santo_domingo.html")
mapa


In [9]:
gdf['poligono'].value_counts()

poligono
Distrito Nacional              501
Santo Domingo Norte             58
Santo Domingo Este              47
Santo Domingo Oeste             27
Los Alcarrizos                  17
Pedro Brand                      5
Fuera de cualquier polígono      1
Name: count, dtype: int64

In [10]:
gdf_group = gdf[['poligono', 'id']].groupby('poligono').agg({'id':'count'}) 
gdf_group['%'] = (gdf[['poligono', 'id']].groupby('poligono').agg({'id':'count'}) /
gdf['id'].count() * 100).round(2)
gdf_group = gdf_group.rename(columns={'id':'qty'})
gdf_group.sort_values(by=['qty'], ascending=False)

,qty,%
poligono,,
Distrito Nacional,501,76.37
Santo Domingo Norte,58,8.84
Santo Domingo Este,47,7.16
Santo Domingo Oeste,27,4.12
Los Alcarrizos,17,2.59
Pedro Brand,5,0.76
Fuera de cualquier polígono,1,0.15
